In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
S3_PATH_EMOTION = {
    'multinomial': {
        'model': 'v34/emotion/multinomial.pkl',
        'vector': 'v34/emotion/tfidf.pkl',
        'bpe': 'v34/emotion/bpe.model',
    },
    'bert': {
        'model': 'v34/emotion/bert-base-emotion.pb',
        'vocab': 'tokenizer/sp10m.cased.bert.vocab',
        'tokenizer': 'tokenizer/sp10m.cased.bert.model',
    },
    'tiny-bert': {
        'model': 'v34/emotion/tiny-bert-emotion.pb',
        'vocab': 'tokenizer/sp10m.cased.bert.vocab',
        'tokenizer': 'tokenizer/sp10m.cased.bert.model',
    },
    'albert': {
        'model': 'v34/emotion/albert-base-emotion.pb',
        'vocab': 'tokenizer/sp10m.cased.v10.vocab',
        'tokenizer': 'tokenizer/sp10m.cased.v10.model',
    },
    'tiny-albert': {
        'model': 'v34/emotion/albert-tiny-emotion.pb',
        'vocab': 'tokenizer/sp10m.cased.v10.vocab',
        'tokenizer': 'tokenizer/sp10m.cased.v10.model',
    },
    'xlnet': {
        'model': 'v34/emotion/xlnet-base-emotion.pb',
        'vocab': 'tokenizer/sp10m.cased.v9.vocab',
        'tokenizer': 'tokenizer/sp10m.cased.v9.model',
    },
    'alxlnet': {
        'model': 'v34/emotion/alxlnet-base-emotion.pb',
        'vocab': 'tokenizer/sp10m.cased.v9.vocab',
        'tokenizer': 'tokenizer/sp10m.cased.v9.model',
    },
}

In [3]:
for k in S3_PATH_EMOTION.keys():
    if k != 'multinomial':
        print(k)
        os.system(f"wget https://f000.backblazeb2.com/file/malaya-model/{S3_PATH_EMOTION[k]['model']}")

bert
tiny-bert
albert
tiny-albert
xlnet
alxlnet


In [4]:
import tensorflow as tf
from tensorflow.tools.graph_transforms import TransformGraph
from glob import glob
tf.set_random_seed(0)

In [5]:
pbs = glob('*.pb')
pbs

['bert-base-emotion.pb',
 'xlnet-base-emotion.pb',
 'alxlnet-base-emotion.pb',
 'albert-base-emotion.pb',
 'tiny-bert-emotion.pb',
 'albert-tiny-emotion.pb']

In [13]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_constants(ignore_errors=true)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-10, fallback_max=10)',
             'quantize_nodes(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

for pb in pbs:
    input_graph_def = tf.GraphDef()
    with tf.gfile.FastGFile(pb, 'rb') as f:
        input_graph_def.ParseFromString(f.read())
    
    if 'bert' in pb:
        inputs = ['Placeholder', 'Placeholder_1']
    if 'xlnet'in pb:
        inputs = ['Placeholder', 'Placeholder_1', 'Placeholder_2']
        
    print(pb, inputs)
    
    transformed_graph_def = TransformGraph(input_graph_def, 
                                           inputs,
                                           ['logits', 'logits_seq'], transforms)
    
    with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
        f.write(transformed_graph_def.SerializeToString())

bert-base-emotion.pb ['Placeholder', 'Placeholder_1']
xlnet-base-emotion.pb ['Placeholder', 'Placeholder_1', 'Placeholder_2']
alxlnet-base-emotion.pb ['Placeholder', 'Placeholder_1', 'Placeholder_2']
albert-base-emotion.pb ['Placeholder', 'Placeholder_1']
tiny-bert-emotion.pb ['Placeholder', 'Placeholder_1']
albert-tiny-emotion.pb ['Placeholder', 'Placeholder_1']


In [7]:
def load_graph(frozen_graph_filename, **kwargs):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # https://github.com/onnx/tensorflow-onnx/issues/77#issuecomment-445066091
    # to fix import T5
    for node in graph_def.node:
        if node.op == 'RefSwitch':
            node.op = 'Switch'
            for index in xrange(len(node.input)):
                if 'moving_' in node.input[index]:
                    node.input[index] = node.input[index] + '/read'
        elif node.op == 'AssignSub':
            node.op = 'Sub'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
        elif node.op == 'AssignAdd':
            node.op = 'Add'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
        elif node.op == 'Assign':
            node.op = 'Identity'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
            if 'validate_shape' in node.attr:
                del node.attr['validate_shape']
            if len(node.input) == 2:
                node.input[0] = node.input[1]
                del node.input[1]

    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [19]:
g = load_graph('xlnet-base-emotion.pb.quantized')
x = g.get_tensor_by_name('import/Placeholder:0')
x_len = g.get_tensor_by_name('import/Placeholder_1:0')
x_len2 = g.get_tensor_by_name('import/Placeholder_2:0')
logits = g.get_tensor_by_name('import/logits:0')

In [20]:
# x, x_len, logits

In [21]:
test_sess = tf.InteractiveSession(graph = g)

In [22]:
%%time
test_sess.run(logits, feed_dict = {x: [[1,2,3,3,4]], x_len: [[1,1,1,1,1]],
                                  x_len2: [[1,1,1,1,1]]})

CPU times: user 2.54 s, sys: 187 ms, total: 2.72 s
Wall time: 1.8 s


array([[-1.9607849, -3.294118 , -2.1960788, 10.039215 ,  5.1764708,
         3.7647057]], dtype=float32)

In [23]:
# %%time
# test_sess.run(logits, feed_dict = {x: [[1,2,3,3,4]], x_len: [[1,1,1,1,1]]})

In [24]:
quantized = glob('*.pb.quantized')
quantized

['bert-base-emotion.pb.quantized',
 'albert-base-emotion.pb.quantized',
 'xlnet-base-emotion.pb.quantized',
 'tiny-bert-emotion.pb.quantized',
 'alxlnet-base-emotion.pb.quantized',
 'albert-tiny-emotion.pb.quantized']

In [25]:
from b2sdk.v1 import *
info = InMemoryAccountInfo()
b2_api = B2Api(info)
application_key_id = 'd3c416cf4cb1'
application_key = '0007c73b0ef09cbff76ebdd5b14f2e0044d6d44b74'
b2_api.authorize_account("production", application_key_id, application_key)
file_info = {'how': 'good-file'}
b2_bucket = b2_api.get_bucket_by_name('malaya-model')

In [26]:
for file in quantized:
    print(file)
    key = file
    outPutname = f"v40/emotion/{file}"
    b2_bucket.upload_local_file(
        local_file=key,
        file_name=outPutname,
        file_infos=file_info,
    )

bert-base-emotion.pb.quantized
albert-base-emotion.pb.quantized
xlnet-base-emotion.pb.quantized
tiny-bert-emotion.pb.quantized
alxlnet-base-emotion.pb.quantized
albert-tiny-emotion.pb.quantized


In [27]:
!rm *.pb*

In [ ]:
# converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
#     graph_def_file='test.pb',
#     input_arrays=['Placeholder', 'Placeholder_1'],
#     input_shapes={'Placeholder' : [None, 512], 'Placeholder_1': [None, 512]},
#     output_arrays=['logits'],
# )
# # converter.allow_custom_ops=True

In [ ]:
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
# converter.target_spec.supported_types = [tf.float16]
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.experimental_new_converter = True
# tflite_model = converter.convert()

In [ ]:
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, 
#                                        tf.lite.OpsSet.SELECT_TF_OPS]
# converter.target_spec.supported_types = [tf.float16]
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# tflite_model = converter.convert()

# with open('tiny-bert-sentiment-float16.tflite', 'wb') as f:
#     f.write(tflite_model)

In [ ]:
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, 
#                                        tf.lite.OpsSet.SELECT_TF_OPS]
# converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
# tflite_model = converter.convert()

# with open('tiny-bert-sentiment-hybrid.tflite', 'wb') as f:
#     f.write(tflite_model)

In [ ]:
# interpreter = tf.lite.Interpreter(model_path='tiny-bert-sentiment-hybrid.tflite')
# interpreter.allocate_tensors()